In [1]:
import numpy as np
#pd untuk melakukan pembacaan data
import pandas as pd

In [2]:
#membaca data csv yang sudah di import
data = pd.read_csv('smsspamcollection.tsv', sep = '\t')
#melihat 5 data teratas dari data yang sudah dibaca
data.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [3]:
#membuat fitur, x merupakan fitur untuk latihan dan y merupakan fitur untuk hasil
x = data['message']
y = data['label']

In [4]:
#untuk import library train_test_split untuk membantu melakukan split data menjadi training dan testing
from sklearn.model_selection import train_test_split

In [5]:
#proses pembagian data, disini data yang diambil untuk testing sebanyak 33%, random state yaitu sama seperti set seed (dalam R) yaitu agar data yang di split menjadi train dan test tidak berubah ubah atau tetap sama
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [8]:
#countVectorizer digunakan untuk mengubah teks tertentu menjadi vektor berdasarkan frekuensi (hitungan) setiap kata yang muncul di seluruh teks
#atau untuk mengekstrak data dari text yang diberikan
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
countV = CountVectorizer()

In [10]:
#mengekstrak data yang ada dalam x_train menggunakan library CountVectorizer, mengubah data text yang ada dalam x_train menjadi matriks
x_train_count = countV.fit_transform(x_train)

In [23]:
x_train_count

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [24]:
x_train.shape

(3733,)

In [25]:
#TfidfVectorizer untuk melihat banyaknya kata kata yang berulang ulng dalam dokumen
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer()

In [27]:
x_train_tfidf = vectorizer.fit_transform(x_train)

In [28]:
x_train_tfidf

<3733x7082 sparse matrix of type '<class 'numpy.float64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [29]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(x_train_tfidf, y_train)

LinearSVC()

In [30]:
x_test_tfidf = vectorizer.fit_transform(x_test)

In [31]:
x_test_tfidf

<1839x4695 sparse matrix of type '<class 'numpy.float64'>'
	with 24177 stored elements in Compressed Sparse Row format>

In [32]:
#buat set prediksi
prediction = clf.predict(x_test_tfidf)

ValueError: X has 4695 features, but LinearSVC is expecting 7082 features as input.

In [33]:
#menggunakan pipeline agar dapat menggunakan clf untuk x_test_tfidf
from sklearn.pipeline import Pipeline

In [34]:
test_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [35]:
test_clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [36]:
prediction = test_clf.predict(x_test)

In [37]:
from sklearn import metrics

#membuat confusion matriks
print(metrics.confusion_matrix(y_test,prediction))

#print classification report
print(metrics.classification_report(y_test,prediction))

#print akurasi
print(metrics.accuracy_score(y_test,prediction))

[[1586    7]
 [  12  234]]
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839

0.989668297988037


In [38]:
test_clf.predict(["Congratulations! You are the winner! Call our number 12345678 to get your price RIGHT now!"])

array(['spam'], dtype=object)

In [39]:
test_clf.predict(["i love you"])

array(['ham'], dtype=object)

In [40]:
test_clf.predict(["Hey please call me now, i need to talk to you"])

array(['ham'], dtype=object)